In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)

In [3]:
df_2020 = pd.read_csv('../../dobrano_2020_pre_preprocessado.csv', sep=';', low_memory=False, encoding='latin1')
df_2020.shape

(454432, 17)

In [4]:
df_2020.head()

,idade,sexo,raca_cor,escolaridade,local_ocorrencia,teve_assistencia_medica,dia_nasc,mes_nasc,ano_nasc,dia_obito,mes_obito,ano_obito,estado,grupo_causa_basica,causa_basica,ocupacao,grupo_ocupacao
0,68.0,M,BRANCA,EF_I,DOMICILIO,NAO,10,5.0,1951.0,25,1,2020,MG,Causas mal definidas e desconhecidas de mortal...,Outras causas mal definidas e as não especific...,NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
1,53.0,M,PRETA,EF_I,DOMICILIO,NAO,16,3.0,1966.0,25,1,2020,MG,Causas mal definidas e desconhecidas de mortal...,Outras causas mal definidas e as não especific...,Trabalhador agropecuário em geral,"TRABALHADORES AGROPECUÁRIOS, FLORESTAIS E DA P..."
2,82.0,F,BRANCA,EF_I,HOSPITAL,NaN,10,7.0,1937.0,14,1,2020,MG,Outras doenças dos intestinos,"Doença diverticular do intestino, de localizaç...",NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
3,73.0,F,BRANCA,EF_I,HOSPITAL,SIM,2,11.0,1946.0,20,1,2020,MG,Outras doenças do aparelho respiratório,Insuficiência respiratória aguda,NaN,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
4,60.0,M,BRANCA,SUPERIOR_C,HOSPITAL,SIM,12,9.0,1959.0,26,1,2020,MG,Outras doenças bacterianas,Septicemia não especificada,Professor da educação de jovens e adultos do ...,PROFISSIONAIS DAS CIÊNCIAS E DAS ARTES ...


#### TRANSFORMANDO DADOS EM NUMÉRICOS

In [6]:
df_2020['raca_cor'] = df_2020['raca_cor'].replace(['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDIGENA'], [1, 2, 3, 4, 5])

In [7]:
df_2020['escolaridade'] = df_2020['escolaridade'].replace(
                                          ['SEM', 'EF_I', 'EF_II', 'EM', 'SUPERIOR_I', 'SUPERIOR_C', 'I'],
                                        [0, 1, 2, 3, 4, 5, 9]
                                         )

In [8]:
df_2020['local_ocorrencia'] = df_2020['local_ocorrencia'].replace(['HOSPITAL', 'OES', 'DOMICILIO', 'VIAP', 'OUTROS', 'I'],
                                            [1, 2, 3, 4, 5, 9]
                                         )

In [9]:
df_2020['teve_assistencia_medica'] = df_2020['teve_assistencia_medica'].replace(['SIM', 'NAO', 'I'], [1, 2, 9]
                                             )

In [10]:
df_2020['sexo'] = df_2020['sexo'].replace(['M', 'F', 'I'], [1, 2, 0]) 

In [11]:
df_cid_grupos = pd.read_csv('../../CID-10-GRUPOS.csv', header=0, sep=';', encoding='latin1')
# df_cid_grupos.head()

df_cid_grupos = df_cid_grupos.drop(columns=['Unnamed: 0'])

In [15]:
df_cid_grupos.head()

,CATINIC,CATFIM,DESCRICAO,DESCRABREV,VALOR
0,A00,A09,Doenças infecciosas intestinais,Doenças infecciosas intestinais,0
1,A15,A19,Tuberculose,Tuberculose,1
2,A20,A28,Algumas doenças bacterianas zoonóticas,Algumas doenças bacterianas zoonóticas,2
3,A30,A49,Outras doenças bacterianas,Outras doenças bacterianas,3
4,A50,A64,Infecções de transmissão predominantemente sexual,Infecções de transmissão predominantemente sexual,4


In [13]:
cid_grupos = [i for i in range(df_cid_grupos.shape[0])]

In [14]:
df_cid_grupos['VALOR'] = cid_grupos

In [24]:
df_ocup_grupo = pd.read_csv('../../Downloads/CBO2002_Grande_Grupo.csv', header=0, sep=';', encoding='latin1')

In [32]:
df_ocup_grupo.head()

,CODIGO,TITULO,VALOR
0,0,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI...",0
1,1,"MEMBROS SUPERIORES DO PODER PÚBLICO, DIRIGENTE...",1
2,2,PROFISSIONAIS DAS CIÊNCIAS E DAS ARTES ...,2
3,3,TÉCNICOS DE NIVEL MÉDIO ...,3
4,4,TRABALHADORES DE SERVIÇOS ADMINISTRATIVOS ...,4


In [27]:
ocup_grupos = [i for i in range(df_ocup_grupo.shape[0])]
ocup_grupos.append(len(ocup_grupos) + 1)

In [28]:
df_ocup_grupo['VALOR'] = ocup_grupos

In [33]:
def group_ocup(row, grupo):
    grupo.loc[(grupo.grupo_ocupacao == row.TITULO),
             ['grupo_ocupacao']] = row.VALOR

In [34]:
df_ocup_grupo.apply(group_ocup, grupo=df_2020, axis='columns')

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [19]:
def categoria_cid(row, cids):
    cids.loc[(cids.grupo_causa_basica == row.DESCRICAO),
             ['grupo_causa_basica']] = row.VALOR

In [20]:
df_cid_grupos.apply(categoria_cid, cids=df_2020, axis='columns')

0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
10     None
11     None
12     None
13     None
14     None
15     None
16     None
17     None
18     None
19     None
20     None
21     None
22     None
23     None
24     None
25     None
26     None
27     None
28     None
29     None
30     None
31     None
32     None
33     None
34     None
35     None
36     None
37     None
38     None
39     None
40     None
41     None
42     None
43     None
44     None
45     None
46     None
47     None
48     None
49     None
50     None
51     None
52     None
53     None
54     None
55     None
56     None
57     None
58     None
59     None
60     None
61     None
62     None
63     None
64     None
65     None
66     None
67     None
68     None
69     None
70     None
71     None
72     None
73     None
74     None
75     None
76     None
77     None
78     None
79     None
80     None
81     None
82     None
83  

In [35]:
df_2020.head()

,idade,sexo,raca_cor,escolaridade,local_ocorrencia,teve_assistencia_medica,dia_nasc,mes_nasc,ano_nasc,dia_obito,mes_obito,ano_obito,estado,grupo_causa_basica,causa_basica,ocupacao,grupo_ocupacao
0,68.0,1,1.0,1.0,3,2.0,10,5.0,1951.0,25,1,2020,MG,201,Outras causas mal definidas e as não especific...,NaN,9
1,53.0,1,2.0,1.0,3,2.0,16,3.0,1966.0,25,1,2020,MG,201,Outras causas mal definidas e as não especific...,Trabalhador agropecuário em geral,6
2,82.0,2,1.0,1.0,1,NaN,10,7.0,1937.0,14,1,2020,MG,117,"Doença diverticular do intestino, de localizaç...",NaN,9
3,73.0,2,1.0,1.0,1,1.0,2,11.0,1946.0,20,1,2020,MG,111,Insuficiência respiratória aguda,NaN,9
4,60.0,1,1.0,5.0,1,1.0,12,9.0,1959.0,26,1,2020,MG,3,Septicemia não especificada,Professor da educação de jovens e adultos do ...,2


In [36]:
data_estados = pd.read_csv('../../Downloads/estados_ibge.csv', sep=';', header=0, encoding='latin1')

In [37]:
data_estados.head()

,CODIGO,UF,ESTADO
0,12,AC,Acre
1,27,AL,Alagoas
2,13,AM,Amazonas
3,16,AP,Amap
4,29,BA,Bahia


In [ ]:
uf = pd.Series(data_estados['UF'].astype('str'))
uf = uf.values

In [38]:
df_2020['estado'] = df_2020['estado'].replace(data_estados['UF'].values, data_estados['CODIGO'].values)

In [39]:
df_2020.head()

,idade,sexo,raca_cor,escolaridade,local_ocorrencia,teve_assistencia_medica,dia_nasc,mes_nasc,ano_nasc,dia_obito,mes_obito,ano_obito,estado,grupo_causa_basica,causa_basica,ocupacao,grupo_ocupacao
0,68.0,1,1.0,1.0,3,2.0,10,5.0,1951.0,25,1,2020,31,201,Outras causas mal definidas e as não especific...,NaN,9
1,53.0,1,2.0,1.0,3,2.0,16,3.0,1966.0,25,1,2020,31,201,Outras causas mal definidas e as não especific...,Trabalhador agropecuário em geral,6
2,82.0,2,1.0,1.0,1,NaN,10,7.0,1937.0,14,1,2020,31,117,"Doença diverticular do intestino, de localizaç...",NaN,9
3,73.0,2,1.0,1.0,1,1.0,2,11.0,1946.0,20,1,2020,31,111,Insuficiência respiratória aguda,NaN,9
4,60.0,1,1.0,5.0,1,1.0,12,9.0,1959.0,26,1,2020,31,3,Septicemia não especificada,Professor da educação de jovens e adultos do ...,2


In [40]:
df_2020 = df_2020.drop(columns=['causa_basica', 'ocupacao'])

#### SEPRANDO DATASETS

In [41]:
df_2020 = df_2020.loc[df_2020['grupo_causa_basica'].notnull()]

In [63]:
df_2020.isnull().sum()

idade                      0
sexo                       0
raca_cor                   0
escolaridade               0
local_ocorrencia           0
teve_assistencia_medica    0
dia_nasc                   0
mes_nasc                   0
ano_nasc                   0
dia_obito                  0
mes_obito                  0
ano_obito                  0
estado                     0
grupo_causa_basica         0
grupo_ocupacao             0
dtype: int64

In [53]:
df_2020 = df_2020.loc[df_2020.teve_assistencia_medica.notnull()]

In [55]:
df_2020 = df_2020.loc[df_2020.raca_cor.notnull()]

In [58]:
df_2020 = df_2020.loc[df_2020.escolaridade.notnull()]

In [61]:
df_2020 = df_2020.loc[df_2020.mes_nasc.notnull()]

In [62]:
df_2020 = df_2020.loc[df_2020.ano_nasc.notnull()]

In [73]:
y = df_2020['grupo_causa_basica'].values

In [72]:
type(y)

pandas.core.series.Series

In [77]:
x = df_2020.drop(columns=['grupo_causa_basica'])

In [78]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [79]:
x_train.shape

(138434, 14)

In [85]:
type(y_train)

numpy.ndarray

In [80]:
y_test.shape

(46145,)

In [81]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [82]:

rf.fit(x_train, y_train)

ValueError: Unknown label type: 'unknown'